# Assignment 15 – Intro to Neural Networks with NumPy

This notebook condenses `numpy_advanced_intro_to_neural_nets.md` into a structured workspace. Work through each exercise in order: first understand the idea, then implement it, and finally reflect on the takeaways.

## Workflow
1. **Research (15–30 min):** clarify the concept, its purpose, the math intuition, and look for visuals.
2. **Implementation (20–30 min):** write the code only after you know what you are doing; comment on intent and check matrix shapes.
3. **Reflection (10–15 min):** record what you learned, open questions, and any "aha" moments.

Recommended resources: 3Blue1Brown for intuition, Papers with Code for practice, and the NumPy docs for vectorization tips. Avoid blind copy/paste, iterate in small steps, and validate each change.

## Exercise I – Single ReLU Neuron
- Implement `NeuronSimplu` with random weights/bias (seeded for reproducibility).
- Add `relu(x) = max(0, x)` and use it inside `forward` to compute `ReLU(weights · inputs + bias)`.
- Print the parameters and output for a sample input, plus any helpful shape checks.
- In markdown, explain why ReLU adds non-linearity, mention pitfalls such as the dying-ReLU issue, and describe the intuition behind weights and bias.

In [ ]:
import random

class SimpleNeuron:
    def __init__(self):
        random.seed(42)

        # weights representing influence of last layer on neuron
        self.weights = [random.uniform(0, 1) for _ in range(4)]
        self.bias = 0

    def relu(self, x):
        return max(0, x)

    def forward(self, x):
        z = self.weights @ x + self.bias

In [31]:
import random
import numpy as np

class Layer:
    def __init__(self, n_in, n_out, use_relu=True):
        random.seed(42)

        self.W = np.random.uniform(-1, 1, (n_out, n_in))
        self.b = np.random.uniform(-1, 1, (n_out,))
        self.use_relu = use_relu
        self.a = np.zeros(n_out)

    def relu(self, z):
        return np.maximum(0, z)
    
    def forward(self, x):
        z = self.W @ x + self.b
        self.a = self.relu(z) if self.use_relu else z
        return self.a

class Layers:
    def __init__(self):
        self.layers: list[Layer] = []

    def add(self, n_in, n_out, use_relu=True):
        self.layers.append(Layer(n_in, n_out, use_relu))

    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x

# Test

import numpy as np

def make_data(n=1000):
    X = np.random.uniform(-1, 1, (n, 2))
    y = X[:, 0] + X[:, 1]
    return X, y

def train(net, X, y, lr=0.01, epochs=200):
    for _ in range(epochs):
        for xi, yi in zip(X, y):
            out = net.forward(xi)

            error = out - yi   # (1,)

            layer = net.layers[-1]

            # derivative of activation
            if layer.use_relu:
                dz = (layer.a > 0).astype(float) * error
            else:
                dz = error  # linear output

            # update
            layer.W -= lr * dz[:, None] * xi[None, :]
            layer.b -= lr * dz

net = Layers()
net.add(2, 1, use_relu=False)

X, y = make_data()
# print([round(float(x), 5) for x in y])
train(net, X, y)

a = random.uniform(-1, 1)
b = random.uniform(-1, 1)

test = np.array([a, b])
print("pred:", net.forward(test))
print("true:", a + b)

pred: [-0.67112489]
true: -0.6711248926388986


# Simple Network Test

In [23]:
import numpy as np

def make_data(n=1000):
    X = np.random.uniform(-1, 1, (n, 2))
    y = X[:, 0] + X[:, 1]
    return X, y

def train(net: Layers, X, y, lr=0.01, epochs=200):
    for _ in range(epochs):
        for xi, yi in zip(X, y):
            # forward
            out = net.forward(xi)

            # compute error (simple squared loss)
            error = out - yi           # shape (1,)
            
            # gradient w.r.t z (ReLU derivative)
            dz = (net.layers[-1].a > 0).astype(float) * error

            # update weights: W -= lr * dz * x
            layer = net.layers[-1]
            layer.W -= lr * dz[:, None] * xi[None, :]
            layer.b -= lr * dz

net = Layers()
net.add(2, 1)

X, y = make_data()
# print([round(float(x), 5) for x in y])
train(net, X, y)

test = np.array([0.3, 0.4])
print("pred:", net.forward(test))
print("true:", 0.3 + 0.4)

pred: [0.7]
true: 0.7


## Exercise II – Sigmoid on Random Data
- Generate 100 random real values, apply a vectorized sigmoid, and store both the raw and transformed arrays.
- Report mean/min/max before and after sigmoid and compare the histograms to show how the function squeezes values into (0, 1).
- Document where sigmoid shines (binary classification, probability outputs), when it struggles (vanishing gradients), and how it differs from tanh or softmax.

## Exercise III – Two-Layer Vectorized Network
- Build `ReteasNeuronala` with dense layers input→hidden→output, ReLU in the hidden layer, sigmoid in the output layer.
- Keep everything vectorized: `Z1 = X · W1 + b1`, `A1 = ReLU(Z1)`, `Z2 = A1 · W2 + b2`, `A2 = sigmoid(Z2)`, `prezice()` returns `(A2 > 0.5).astype(int)`.
- Test with synthetic data, print parameter and activation shapes, and prove the batch processing works.
- Note current limits (no training/backprop yet, hidden size is a manual choice) and how you would extend this skeleton to deeper nets.

## Evaluation & Good Practices
- **Correctness (40%)** – math functions and matrix ops behave as intended.
- **Vectorization (30%)** – NumPy handles the heavy lifting; no unnecessary loops.
- **Clarity (20%)** – meaningful names plus comments that explain intent.
- **Testing (10%)** – run sample inputs, print shapes/results, and check reproducibility.

Always log new questions, seed randomness when needed, and use shape prints to debug quickly.